In [168]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
batting = pd.read_csv(r'C:\Users\Chris Puglisi\Data Science Projects\Baseball Betting Hedge Fund\baseballdatabank-master\core\Batting.csv')
teams = pd.read_csv(r'C:\Users\Chris Puglisi\Data Science Projects\Baseball Betting Hedge Fund\baseballdatabank-master\core\Teams.csv')
pitching = pd.read_csv(r'C:\Users\Chris Puglisi\Data Science Projects\Baseball Betting Hedge Fund\baseballdatabank-master\core\Pitching.csv')

In [4]:
teams_2003_17 = teams[teams['yearID']> 1920]
RS_test_train = teams_2003_17[['yearID','teamID','lgID','divID','Ghome','W','R','AB','H','2B','3B','HR','SB','BB','RA','HA','HRA','BBA','SOA','E','FP']]

RS_test_train['teamID'] = RS_test_train['teamID'] + RS_test_train['yearID'].map(str)

RS_test_train = RS_test_train.set_index('teamID')

C:\Users\Chris Puglisi\Anacondathreefive\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
df_2004=RS_test_train.loc[RS_test_train['yearID']== 2004]
df_2005=RS_test_train.loc[RS_test_train['yearID']== 2005]
df_2006=RS_test_train.loc[RS_test_train['yearID']== 2006]
df_2007=RS_test_train.loc[RS_test_train['yearID']== 2007]
df_2008=RS_test_train.loc[RS_test_train['yearID']== 2008]
df_2009=RS_test_train.loc[RS_test_train['yearID']== 2009]
df_2010=RS_test_train.loc[RS_test_train['yearID']== 2010]
df_2011=RS_test_train.loc[RS_test_train['yearID']== 2011]
df_2012=RS_test_train.loc[RS_test_train['yearID']== 2012]
df_2013=RS_test_train.loc[RS_test_train['yearID']== 2013]
df_2014=RS_test_train.loc[RS_test_train['yearID']== 2014]
df_2015=RS_test_train.loc[RS_test_train['yearID']== 2015]
df_2016=RS_test_train.loc[RS_test_train['yearID']== 2016]
df_2017=RS_test_train.loc[RS_test_train['yearID']== 2017]

In [6]:
RS_test_train_yrdrop = RS_test_train.drop('yearID', 1)
#RS_test_train_yrdrop = RS_test_train_yrdrop.drop('W', 1)
RS_ML = RS_test_train_yrdrop

RS_ML['lgdivID'] = RS_ML['lgID'] + RS_ML['divID']
RS_ML = RS_ML.drop(['lgID','divID'], axis = 1)

In [7]:
RS_ML['lgdivID'] = RS_ML['lgdivID'].astype('category')
RS_ML['lgdivID'] = RS_ML['lgdivID'].cat.codes

In [8]:
train, test = train_test_split(RS_ML, test_size = 0.15)

train_x = train.drop('W', 1)
train_y = train['W']

test_x = test.drop('W', 1)
test_y = test['W']

In [9]:
tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100)

tree.fit(train_x, train_y)

tree_predictions = tree.predict(test_x)

In [10]:
cross_val_score(tree, test.drop('W',1), test['W'], cv = 5)

array([0.77789816, 0.7623852 , 0.5735478 , 0.48802781, 0.55716134])

In [11]:
gain_tree1 = pd.DataFrame({'Gain': tree.feature_importances_}, index = train_x.columns).sort_values('Gain', ascending = False)

In [12]:
#pd.crosstab(test_y, tree_predictions, rownames=['Actual'], colnames = ['Predicted:'], margins = True)

In [13]:
df_top5 = RS_ML[['W','RA','R','BB','HR','H']]

In [14]:
traintop5, testtop5 = train_test_split(df_top5, test_size = 0.15)

traintop5_x = traintop5.drop('W', 1)
traintop5_y = traintop5['W']

testtop5_x = testtop5.drop('W', 1)
testtop5_y = testtop5['W']

In [15]:
tree5 = DecisionTreeRegressor(criterion = 'mse', max_depth = 10)

tree5.fit(traintop5_x, traintop5_y)

tree5_predictions = tree5.predict(testtop5_x)

In [16]:
cross_val_score(tree, test.drop('W',1), test['W'], cv = 5)

array([0.64262498, 0.71465528, 0.6006179 , 0.53105226, 0.60177768])

# Picking the Right Model


In [154]:
#lets use the high feature data fisrt
!pip3 install xgboost

In [155]:
from sklearn import ensemble, svm, linear_model
from sklearn.svm import SVR
from sklearn.linear_model import ridge_regression, Lasso
from sklearn import model_selection
import xgboost
from xgboost import XGBRegressor
import warnings
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
warnings.filterwarnings('ignore')

In [157]:
MLA = [
    
    #Ensemble Methods
    ensemble.AdaBoostRegressor(),
    ensemble.BaggingRegressor(),
    #ensemble.ExtraTreesClassifier(),
    #ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestRegressor(),
    
    #Support Vector Regression
    svm.SVR(kernel ='rbf'),
    #svm.SVR(kernel = 'linear'),
    #svm.SVR(kernel = 'poly'),
    
    #GLM
    linear_model.LinearRegression(),
    linear_model.LogisticRegression(),
    linear_model.Ridge(),
    linear_model.Lasso(),
    linear_model.ElasticNet(),
    
    #XGBR
    XGBRegressor()
    
]

In [158]:
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = 0.3, train_size = .6, random_state = 0)
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

MLA_compare

,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time


In [162]:
Target = ['W']
MLA_predict = RS_ML[Target]
x_bin = ['Ghome','R','AB','H','2B','3B','HR','SB','BB','RA','HA','HRA','BBA','SOA','E','FP']
x_bin2 = ['R','RA']

In [163]:
row_index = 0
for alg in MLA:
    
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    cv_results = model_selection.cross_validate(alg, RS_ML[x_bin2], RS_ML[Target], cv = cv_split)
    
    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3
    
    alg.fit(RS_ML[x_bin2], RS_ML[Target])
    MLA_predict[MLA_name] = alg.predict(RS_ML[x_bin2])
    
    row_index+=1
    

    
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
warnings.filterwarnings('ignore')
MLA_compare

,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
9,XGBRegressor,"{'base_score': 0.5, 'booster': 'gbtree', 'cols...",0.884426,0.827455,0.0450265,0.054452
4,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.807874,0.795996,0.0559089,0.00199471
6,Ridge,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",0.807874,0.795996,0.0559089,0.00169387
8,ElasticNet,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",0.807873,0.795992,0.0558669,0.00129597
7,Lasso,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",0.807872,0.795989,0.0558314,0.0016952
1,BaggingRegressor,"{'base_estimator': None, 'bootstrap': True, 'b...",0.963729,0.789543,0.0481805,0.0275273
2,RandomForestRegressor,"{'bootstrap': True, 'criterion': 'mse', 'max_d...",0.961795,0.788541,0.0590507,0.0274257
0,AdaBoostRegressor,"{'base_estimator': None, 'learning_rate': 1.0,...",0.78876,0.759355,0.0549408,0.0797927
5,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.0841492,0.0631988,0.0253486,0.103621
3,SVR,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd...",0.118854,0.00109184,0.0123829,0.138303


# Best of the Best

In [164]:
lnr = linear_model.LinearRegression()

scores = cross_val_score(lnr, RS_ML[x_bin2], RS_ML[Target], cv = 5)

print(scores)
print()
print('Average score:', np.mean(scores))

y_predlnr = cross_val_predict(lnr, RS_ML[x_bin2], RS_ML[Target], cv = 5)
#conf_mat = confusion_matrix(RS_ML[Target], y_pred)

[0.82591443 0.8527584  0.67749119 0.71044006 0.80362863]

Average score: 0.7740465416312939


In [165]:
rdg = linear_model.Ridge()

scores = cross_val_score(rdg, RS_ML[x_bin2], RS_ML[Target], cv = 5)

print(scores)
print()
print('Average score:', np.mean(scores))

[0.82591444 0.8527584  0.67749118 0.71044006 0.80362863]

Average score: 0.7740465425619834


In [166]:
eln = linear_model.ElasticNet(alpha = 0.1, l1_ratio = 0.5)

scores = cross_val_score(eln, RS_ML[x_bin2], RS_ML[Target], cv = 5)

print(scores)
print()
print('Average score:', np.mean(scores))

[0.82592548 0.85275412 0.67748583 0.71043936 0.80363012]

Average score: 0.7740469834552762


In [167]:
lso = linear_model.Lasso()

scores = cross_val_score(lso, RS_ML[x_bin2], RS_ML[Target], cv = 5)

print(scores)
print()
print('Average score:', np.mean(scores))

[0.82610268 0.85267473 0.6773955  0.71042821 0.80365292]

Average score: 0.7740508083962124


# Prediction BreakDown

In [81]:
x_bin3 = ['Ghome','R','AB','H','3B','HR','SB','BB','RA','HRA','BBA','SOA']

X_train = RS_ML[x_bin2]
X_train = X_train[:-30]

X_test = df_2017[x_bin2] 

y_train = RS_ML[Target]
y_train = y_train[:-30]
y_test = df_2017[Target]

In [82]:
prediction = []

lnr.fit(X_train, y_train)
win_pred = lnr.predict(X_test)

#prediction['Lin Reg Prediction'] = win_pred

y_preds = y_test

y_preds['W Pred'] = win_pred

y_preds['diff'] = y_preds['W'] - y_preds['W Pred']

clusterOver = y_preds[y_preds['diff'] > 3]
clusterUnder = y_preds[y_preds['diff'] < -3]

y_preds.head()


,W,W Pred,diff
teamID,,,
ARI2017,93,96.427977,-3.427977
ATL2017,72,72.160474,-0.160474
BAL2017,75,70.707768,4.292232
BOS2017,93,93.240532,-0.240532
CHA2017,67,68.477635,-1.477635


In [100]:
rdg.fit(X_train, y_train)
win_predR = rdg.predict(X_test)

#prediction['Lin Reg Prediction'] = win_pred

y_predsR = y_test

y_predsR['W Pred Ridge'] = win_predR

y_predsR['diff'] = y_predsR['W'] - y_predsR['W Pred Ridge']


y_predsR.head()

,W,W Pred,diff,W Pred Ridge
teamID,,,,
ARI2017,93,96.417164,-3.417164,96.417164
ATL2017,72,71.964520,0.035480,71.964520
BAL2017,75,70.856823,4.143177,70.856823
BOS2017,93,93.525863,-0.525863,93.525863
CHA2017,67,68.264320,-1.264320,68.264320


In [83]:
Cluster_Error_occ = len(clusterOver) + len(clusterUnder)

Cluster_Error_occ

11

In [84]:
y_true = y_preds.drop(['W Pred', 'diff'], axis = 1)
mean_squared_error(y_true, win_pred)

15.57023177897852

In [140]:
from sklearn.model_selection import GridSearchCV

rdg.fit(X_train, y_train)
win_predR = rdg.predict(X_test)

#prediction['Lin Reg Prediction'] = win_pred

y_predsR = y_test

y_predsR['W Pred Ridge'] = win_predR

y_predsR['diff'] = y_predsR['W'] - y_predsR['W Pred Ridge']

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

grid = GridSearchCV(estimator=rdg, param_grid=dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.8922626447079337
1.0


In [142]:
eln.fit(X_train, y_train)
win_predE = eln.predict(X_test)

#prediction['Lin Reg Prediction'] = win_pred

y_predsE = y_test

y_predsE['W Pred ElasticNet'] = win_predE

y_predsE['diff'] = y_predsE['W'] - y_predsE['W Pred ElasticNet']

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

grid = GridSearchCV(estimator=eln, param_grid=dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

TypeError: fit() got an unexpected keyword argument 'alpha'

In [85]:
vegas_2017 =  pd.read_csv(r'C:\Users\Chris Puglisi\Data Science Projects\Baseball Betting Hedge Fund\Vegas Odds.csv')
vegas_2017 = vegas_2017.sort_values('teamID')
vegas_2017['teamID'] = vegas_2017['teamID'] + '2017'
vegas_2017 = vegas_2017.set_index('teamID')

vegas_2017['Actual W'] = df_2017['W']
vegas_2017['Lin Reg Pred'] = y_preds['W Pred']

In [86]:
py_pred =  pd.read_csv(r'C:\Users\Chris Puglisi\Data Science Projects\Baseball Betting Hedge Fund\petaPred.csv')

In [87]:
py_pred = py_pred.sort_values('teamID')
py_pred['teamID'] = py_pred['teamID'] + '2017'
py_pred = py_pred.set_index('teamID')


In [88]:
py_pred['petaPred'] = py_pred['petaPred'] * 162


In [106]:
vegas_2017['petaPred'] = py_pred['petaPred']
vegas_2017['Ridge Reg'] = y_predsR['W Pred Ridge']
vegas_2017['Elastic Net'] = y_predsE['W Pred ElasticNet']

vegas_2017.head()

,Vegas W,Actual W,Lin Reg Pred,petaPred,lin per err,peta per err,Ridge Reg,Elastic Net
teamID,,,,,,,,
ARI2017,78.5,93,96.427977,96.287977,3.685996,3.535459,96.417164,96.536876
ATL2017,71.5,72,72.160474,72.526962,0.222881,0.731891,71.964520,72.019768
BAL2017,84.5,75,70.707768,71.856612,-5.722976,-4.191184,70.856823,71.172552
BOS2017,90.5,93,93.240532,92.875626,0.258637,-0.133735,93.525863,94.642333
CHA2017,73.5,67,68.477635,69.974650,2.205425,4.439776,68.264320,68.107533


In [109]:
vegas_2017['lin per err'] =100* ((vegas_2017['Lin Reg Pred']-vegas_2017['Actual W'])/vegas_2017['Actual W'])
vegas_2017['peta per err'] =100* ((vegas_2017['petaPred']-vegas_2017['Actual W'])/vegas_2017['Actual W'])
vegas_2017['Ridge per err'] =100* ((vegas_2017['Ridge Reg']-vegas_2017['Actual W'])/vegas_2017['Actual W'])
vegas_2017['Elast per err'] =100* ((vegas_2017['Elastic Net']-vegas_2017['Actual W'])/vegas_2017['Actual W'])


In [132]:
vegas_2017_perEr = vegas_2017[['lin per err','peta per err','Ridge per err','Elast per err']]

In [133]:
vegas_2017_perEr = vegas_2017_perEr.abs()

In [134]:
vegas_2017_perEr['Best Method'] = vegas_2017_perEr.idxmin(axis=1)

In [137]:
vegas_2017_perEr['Best Method'].value_counts()
#vegas_2017_perEr

peta per err     12
Elast per err     8
lin per err       8
Ridge per err     2
Name: Best Method, dtype: int64

In [92]:
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, lnr.coef_[0][idx]))

The coefficient for Ghome is 0.6575506005539471
The coefficient for R is 0.0897070733952808
The coefficient for AB is 0.0009730627549906385
The coefficient for H is 0.007055258075729879
The coefficient for 2B is -0.006961286638246479
The coefficient for 3B is 0.0057769712683772255
The coefficient for HR is 0.011001845092272236
The coefficient for SB is 0.0046476961843246595
The coefficient for BB is 0.006351079275097367
The coefficient for RA is -0.0963878068350838
The coefficient for HA is 0.0004965357339465916
The coefficient for HRA is -0.007638985930775854
The coefficient for BBA is -0.003839642853437167
The coefficient for SOA is 0.0024110616975730626
The coefficient for E is 0.08447480969519476
The coefficient for FP is 565.4613523703599


In [93]:
#x_bin3 = ['Ghome','R','AB','H','2B','3B','HR','SB','BB','RA','HA','HRA','BBA','SOA','E','FP']
rdg.fit(X_train, y_train)
win_pred = rdg.predict(X_test)

In [94]:
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, rdg.coef_[0][idx]))

The coefficient for Ghome is 0.6993569361499391
The coefficient for R is 0.08882510956183577
The coefficient for AB is 0.0023388174915137078
The coefficient for H is 0.006870559812663055
The coefficient for 2B is -0.007400634248284474
The coefficient for 3B is 0.00513064412959362
The coefficient for HR is 0.01228048231447278
The coefficient for SB is 0.005887126233388014
The coefficient for BB is 0.006677562850302219
The coefficient for RA is -0.09856845513040068
The coefficient for HA is 0.0021617031086006828
The coefficient for HRA is -0.008707710006277946
The coefficient for BBA is -0.0037741767395162644
The coefficient for SOA is 0.0020923912819956264
The coefficient for E is -0.0033434476801975617
The coefficient for FP is 0.3259344370948475


In [95]:
lso.fit(X_train, y_train)
win_pred = lso.predict(X_test)

In [97]:
lso.coef_

array([ 1.32951364e-01,  9.05569952e-02,  1.15022174e-02,  1.31731169e-04,
       -7.20539356e-03,  8.38110592e-04,  5.59069124e-03,  6.98452169e-03,
        7.93649881e-03, -1.02289685e-01,  3.99046891e-03, -1.80439017e-03,
       -3.00400974e-03,  2.39280808e-03, -0.00000000e+00,  0.00000000e+00])